In [75]:
# import libraries

import openai
import os
import glob
from dotenv import load_dotenv, find_dotenv
from langchain.evaluation import load_evaluator
from langchain.evaluation import EvaluatorType
from langchain.llms import OpenAI
from langchain.prompts import PromptTemplate
import gradio as gr

import pandas as pd
import json

In [76]:
import warnings
warnings.filterwarnings("ignore")

In [77]:
# load and read data.json

with open('data.json') as f:
  data = json.load(f)

In [78]:
# load the env.txt that contains the OPENAI key

load_dotenv('env.txt')
openai_api_key = os.getenv('OPENAI_API_KEY')

In [79]:
llm = OpenAI(
    openai_api_key=openai_api_key,
    model ='gpt-3.5-turbo-instruct'
)
textType = None

In [80]:
sample_feedback = "Your narrative effectively captures the tension, excitement, and triumph of your debate team's journey. Here are some strengths: Engaging Opening: The beginning of your story draws the reader in by describing the nervous atmosphere and the stakes involved in the debate competition. It effectively sets the stage for the rest of the narrative. Well-Developed Characters: You provide a glimpse into the personalities and emotions of the characters, particularly the protagonist and their team. The coach's reassurance, the competitors' condescending behavior, and the team's reaction after the win all contribute to a well-rounded story. Effective Use of Descriptive Language: Your use of descriptive language helps paint a vivid picture of the scene. The physical sensations of anxiety, the discomfort of the formal uniform, and the tightness of the tie contribute to the reader's immersion in the story. Narrative Arc and Resolution: The narrative follows a clear arc from nervous anticipation to the surprising victory and the subsequent confrontation with the opposing team. The resolution, where the underdogs stand up for themselves, adds a satisfying conclusion to the story. Themes of Resilience and Triumph: The story effectively conveys themes of resilience, determination, and triumph in the face of adversity. The underdog narrative is a powerful and relatable element that many readers can connect with. Areas for Consideration: Show, Don't Tell: While you do a good job describing the physical sensations and emotions, consider showing more of the characters' internal thoughts and reactions. This can enhance the reader's connection to the characters and their experiences. Dialogue Punctuation: Pay attention to the punctuation of your dialogue. For example, use commas or periods inside quotation marks, and make sure each speaker's dialogue is on a new line. Diverse Sentence Structure: While your sentence structure is generally varied, consider experimenting with different sentence lengths and structures to add more dynamic flow to the narrative. Deepening Emotional Impact: Explore opportunities to deepen the emotional impact of certain moments, such as the announcement of the winner. Delve into the characters' thoughts and feelings to convey the magnitude of their emotions. Overall, your narrative is compelling and successfully conveys the triumph of the underdog team. With a few adjustments, you can enhance certain elements to create an even more immersive and emotionally resonant experience for the reader."

In [81]:
narrative_custom_criterion = {
    "grammar": (
        "Does the essay follow proper grammar rules?"
        "Does the essay follow proper pronunciation rules?"
        "Does the output use a robust dictionary of vocabulary?"
    ),
    "coherence": "Does the output have a coherent flow of ideas?",
    "relevance":"Does the output fully address the task?",
    "description":"Does the output include sensory words?",
    "characters":"Does the output capture the personality or emotions of the characters in the story?",
    "openings": "Does the output have an engaging opening?"
}

In [82]:
argumentative_custom_criterion = {
    "grammar": "Does the output follow proper grammar, spelling, and pronunciation rules?",
    "coherence": "Does the output have a coherent flow of ideas?",
    "vocabulary":"Does the output use a robust dictionary of vocabulary?",
    "relevance":"Does the output fully address the task?",
    "attention":"Does the output capture your attention?",
    
}

In [83]:
def model(source,essay_type,essay):
        if textType.lower() == "narrative":
                custom_criterion = narrative_custom_criterion
        else:
                custom_criterion = narrative_custom_criterion

        evalReso = load_evaluator(
        EvaluatorType.CRITERIA,
        criteria = custom_criterion,
        llm=llm,
        prompt=PromptTemplate.fromTemplate("You are a English O'level Cambridge Grader for argumentative or narrative writing.")#put context in here (e.g. you are...)
        )

        for i in data[essay_type]:
                question = "Question: " + i["question"] + " Answer: " + i["answer"]

                evalResult = evalReso.evaluate_strings(prediction=sample_feedback, input=question)

        evalResult = evalReso.evaluate_strings(prediction=source, input=essay)
        

In [84]:
def eval(source,essay_type,essay):
    source,essay_type,essay = source,essay_type,essay
    model(essay_type)

In [85]:
demo = gr.Interface(
    eval,
    ["text","text","text"],
    "text",
    title = f'O level  Essay Grader'
)
demo.launch()  

Running on local URL:  http://127.0.0.1:7863

To create a public link, set `share=True` in `launch()`.
